### LincKer
This programme identifies neighbouring mRNAs for a list of genes of interest (in this case, lncRNAs) in a GFF file. For each lncRNA, the programme locates up to 5 upstream and downstream neighbours, although this parameter can be adjusted according to user requirements. The output includes the distance between the lncRNA and each neighbouring mRNA, the differentially expressed (DE) status of neighbouring mRNAs, and other relevant information. The files needed to run the notebooks can be found in the notebooks_files folder.

*Executed with Python v3.11.4*

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/yahelGB/M.Sc_thesis/blob/main/notebooks/lncker.ipynb)

In [89]:
# import libraries
import os
from collections import defaultdict

In [ ]:
# initialise a dictionary to store information for each contig
contigs = {}

# load and read GFF file and extract features
with open('/path/to/GCF_003789085.1_ASM378908v1_genomic.gff', 'r') as f_input:
    for line in f_input:
        if line.startswith('#') or not line.strip():
            continue

        # parse the line into parts using the tab delimiter
        parts = line.strip().split('\t')
        contig_id = parts[0]
        feature_type = parts[2]
        start = int(parts[3])
        end = int(parts[4])
        strand = parts[6]
        metadata = parts[8]
        
        # structuring metadata in a dictionary
        metadata_split = metadata.split(';')
        metadata_dict = {item.split('=')[0]: item.split('=')[1] for item in metadata_split if '=' in item}

        # create a feature dictionary
        feature = {
            'type': feature_type,
            'start': start,
            'end': end,
            'strand': strand,
            'metadata': metadata_dict,
        }

        # Add the feature to the corresponding contig
        if contig_id not in contigs:
            contigs[contig_id] = []
        contigs[contig_id].append(feature)

In [ ]:
# specify the path to the file containing the IDs of interest
# note: update 'input_filename' with the correct path and filename each time 
# a new set of lncRNAs is analysed, these files are generated by the notebook 
# 'exclusive_lncRNAs.ipynb'.

input_filename = '/path/to/sample_exclusive_lncRNAs.txt'
ids_interes = set()

# read the IDs from the specified file and store them in a set
with open(input_filename, 'r') as f_ids:
    for line in f_ids:
        ids_interes.add(line.strip())

print('Number of lncRNAs loaded:', len(ids_interes))

In [ ]:
# read the differentially expressed mRNAs and store them in a set
de_mrnas = set()
with open('/path/to/de_mRNAs.txt', 'r') as f_de:
    for line in f_de:
        de_mrnas.add(line.strip())

print('Number of mRNAs DE:', len(de_mrnas))

In [93]:
# create a dictionary to store the contigs containing the IDs of interest
contigs_interes = defaultdict(list)

# check each contig for IDs of interest
for contig_id, features in contigs.items():
    for feature in features:
        if feature['type'] == 'lnc_RNA':
            metadata_dict = feature['metadata']
            gene_id = metadata_dict.get('ID', 'Unknown')
            if gene_id.startswith('rna-'):
                gene_id = gene_id[4:]
            if gene_id in ids_interes:
                contigs_interes[gene_id].append(contig_id)

In [94]:
# main function 
def find_multiple_mRNA_neighbors(features, lncRNA_index, num_neighbors=5):
    """
    search for mRNA neighbours around a given lncRNA, by default, identifies
    up to 5 upstream and 5 downstream neighbours, though the number of neighbours
    can be adjusted according to user requirements.

    only the first mRNA after each gene is considered, ignoring mRNA variants.
    """
    upstream_mRNAs = []
    downstream_mRNAs = []
    lncRNA = features[lncRNA_index]

    # neighbouring gene tracking
    last_gene_upstream = None
    last_gene_downstream = None

    # search up to 5 upstream neighbours
    for i in range(lncRNA_index - 1, -1, -1):
        feature = features[i]
        if feature['type'] == 'gene':
            last_gene_upstream = i
        elif feature['type'] == 'mRNA' and last_gene_upstream is not None:
            upstream_mRNAs.append(i)
            last_gene_upstream = None
            if len(upstream_mRNAs) == num_neighbors:
                break

    # search up to 5 downstream neighbours
    for i in range(lncRNA_index + 1, len(features)):
        feature = features[i]
        if feature['type'] == 'gene':
            last_gene_downstream = i
        elif feature['type'] == 'mRNA' and last_gene_downstream is not None:
            downstream_mRNAs.append(i)
            last_gene_downstream = None
            if len(downstream_mRNAs) == num_neighbors:
                break

    return upstream_mRNAs[:num_neighbors], downstream_mRNAs[:num_neighbors]

In [95]:
# calculate distances and assess for overlapping
def calculate_distance(lncRNA, mRNA):
    if mRNA['end'] < lncRNA['start']:
        return (lncRNA['start'] - mRNA['end']) / 1000
    elif mRNA['start'] > lncRNA['end']:
        return (mRNA['start'] - lncRNA['end']) / 1000
    else:
        overlap_bases = min(lncRNA['end'], mRNA['end']) - max(lncRNA['start'], mRNA['start']) + 1
        return -overlap_bases / 1000

In [ ]:
# write the output
output_filename = os.path.splitext(input_filename)[0] + "_lncker_output.txt"

with open(output_filename, 'w') as f_output:
    # headers
    f_output.write("Contig\tlncRNA_ID\tlncRNA_strand\tmRNA_ID\tmRNA_strand\tDE_status\tmRNA_Position\tdistance_Kb\tproduct\n")

    # search for mRNA neighbours for the IDs of interest and verify their DE status
    for gene_id in ids_interes:
        for contig_id in contigs_interes[gene_id]:
            features = contigs[contig_id]
            for i, feature in enumerate(features):
                if feature['type'] == 'lnc_RNA' and feature['metadata'].get('ID', '').endswith(gene_id):
                    upstream_mRNAs, downstream_mRNAs = find_multiple_mRNA_neighbors(features, i)

                    # process mRNAs upstream
                    for idx, upstream_mRNA_idx in enumerate(upstream_mRNAs):
                        upstream_mRNA = features[upstream_mRNA_idx]
                        upstream_mRNA_id = upstream_mRNA['metadata'].get('ID', 'Unknown').lstrip("rna-")
                        upstream_mRNA_product = upstream_mRNA['metadata'].get('product', 'Unknown')
                        distance_upstream = calculate_distance(feature, upstream_mRNA)
                        de_status_upstream = "DE" if upstream_mRNA_id in de_mrnas else "not DE"

                        # find overlapping genes
                        distance_value = f"{distance_upstream:.3f}"

                        f_output.write(f"{contig_id}\t{gene_id}\t{feature['strand']}\t{upstream_mRNA_id}\t{upstream_mRNA['strand']}\t{de_status_upstream}\t{idx + 1}_U\t{distance_value}\t{upstream_mRNA_product}\n")

                    # process mRNAs downstream
                    for idx, downstream_mRNA_idx in enumerate(downstream_mRNAs):
                        downstream_mRNA = features[downstream_mRNA_idx]
                        downstream_mRNA_id = downstream_mRNA['metadata'].get('ID', 'Unknown').lstrip("rna-")
                        downstream_mRNA_product = downstream_mRNA['metadata'].get('product', 'Unknown')
                        distance_downstream = calculate_distance(feature, downstream_mRNA)
                        de_status_downstream = "DE" if downstream_mRNA_id in de_mrnas else "not DE"

                        # find overlapping genes
                        distance_value = f"{distance_downstream:.3f}"

                        f_output.write(f"{contig_id}\t{gene_id}\t{feature['strand']}\t{downstream_mRNA_id}\t{downstream_mRNA['strand']}\t{de_status_downstream}\t{idx + 1}_D\t{distance_value}\t{downstream_mRNA_product}\n")

print(f"Results have been written to {output_filename}")